In [3]:
import pandas as pd
import numpy as np
import scipy as sp
import miRNET, miRNET_enrichment
Targets = miRNET.Targets(path_to_miRTarBase='./baseData/hsa_miRTarBase.csv')
import warnings
import collections
from matplotlib import pyplot as plt
import random
from scipy.stats import chi2_contingency
import json
import networkx as nx

ModuleNotFoundError: No module named 'miRNET'

In [ ]:
####################
#up-regulated mIRNA#
####################

In [2]:
with open('./Supl/miRNAs_key_genes/up_miRNAs_key_genes_dict.json') as json_file:
    up_dict = json.load(json_file)

NameError: name 'json' is not defined

In [212]:
def grep_target_gene_by_paths(miR_names, miR_dict, Reactome_ID):
    target_gene = list()
    for miR in miR_names:
        target_gene += miR_dict[miR]
    
    enrich_res = miRNET_enrichment.reactome_enrichment(target_gene, species='Homo sapiens')
    enrich_res = miRNET_enrichment.reac_pars(enrich_res)
    G_enrich = miRNET_enrichment.get_net(enrich_res) #граф сигнальных путей
    reactome_df = miRNET_enrichment.dendro_reactome_to_pandas(enrich_res, G_enrich)
    
    pathway_genes = reactome_df.loc[reactome_df.iloc[:,0] == Reactome_ID,].iloc[0,12]
    
    miR_target_pathway_dict = dict((key,[]) for key in miR_names)
    for miR in miR_names:
        for gene in miR_dict[miR]:
            if gene in pathway_genes:
                miR_target_pathway_dict[miR].append(gene)
    
    return miR_target_pathway_dict

In [215]:
TGF_b_miRs = ['miR-93-', 'miR-222-', 'miR-29a-', 'miR-451a', 'miR-497-']
VEGF_miRs = ['miR-221-', 'miR-133a-', 'miR-199a-3p']


TGFb_miR_gene = grep_target_gene_by_paths(TGF_b_miRs, up_dict, 'R-HSA-2173793')
VEGF_miR_gene = grep_target_gene_by_paths(VEGF_miRs, up_dict, 'R-HSA-194138')

In [237]:
def dict_to_edge_table(dct):
    res_df = pd.DataFrame(columns=['miR', 'target'])
    for key in dct.keys():
        for elm in dct[key]:
            res_df = pd.concat([res_df, pd.DataFrame(list(zip([key], [elm])), columns=['miR', 'target'])])
    return res_df

In [240]:
dict_to_edge_table(TGFb_miR_gene).to_csv('./edge_mIR_TGF.csv',index=False, index_label=False, sep=',')
dict_to_edge_table(VEGF_miR_gene).to_csv('./edge_mIR_VEGF.csv',index=False, index_label=False, sep=',')